In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = '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',
    model_name= "llama-3.3-70b-versatile",
)
response = llm.invoke("The first person to land on moon was")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.ycombinator.com/companies/leafpress/jobs/vms9dS0-swe-intern?utm_source=syn_li")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


SWE Intern at Leafpress | Y Combinator



































































































































































































































































AboutWhat Happens at YC?ApplyYC Interview GuideFAQPeopleYC BlogCompaniesStartup DirectoryFounder DirectoryLaunch YCStartup JobsAll Jobs◦ Engineering◦ Operations◦ Marketing◦ SalesStartup Job GuideYC Startup Jobs BlogFind a Co-FounderLibrarySAFEResourcesStartup SchoolNewsletterRequests for StartupsFor InvestorsHacker NewsBookfaceOpen main menuApply for S2025 batch.ApplyLeafpressAI Copilot for Energy DataSWE Intern$3K - $8KLocationSan Francisco, CA, US / Remote (US)Job TypeInternshipVisaUS citizen/visa onlyConnect directly with founders of the best YC-funded startups.Apply to role ›Jianna LiuFounderAbout the roleAbout Leafpress
Leafpress automates utility data collection for large real estate fir

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM THE WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience','skills' and 'description'.
    Only return the valid JSON.
    ###VALID JSON (NO PREAMBLE):
    
    """
)
chain_extract = prompt_template | llm
res = chain_extract.invoke(input={'page_data' : page_data})
print(res.content)

```json
{
  "role": "SWE Intern",
  "experience": "Internship",
  "skills": [
    "Python programming",
    "Web scraping"
  ],
  "description": "Leafpress automates utility data collection for large real estate firms. We do this by extracting data from millions of utility bills at once, automating reporting to sustainability regulators. We’re looking for a skilled SWE intern to assist us in building and integrating Python web scrapers."
}
```


In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res= json_parser.parse(res.content)
json_res

{'role': 'SWE Intern',
 'experience': 'Internship',
 'skills': ['Python programming', 'Web scraping'],
 'description': 'Leafpress automates utility data collection for large real estate firms. We do this by extracting data from millions of utility bills at once, automating reporting to sustainability regulators. We’re looking for a skilled SWE intern to assist us in building and integrating Python web scrapers.'}

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
print(df.head())


                           Techstack                                  Links
0            React, Node.js, MongoDB    https://example.com/react-portfolio
1           Angular,.NET, SQL Server  https://example.com/angular-portfolio
2  Vue.js, Ruby on Rails, PostgreSQL      https://example.com/vue-portfolio
3              Python, Django, MySQL   https://example.com/python-portfolio
4          Java, Spring Boot, Oracle     https://example.com/java-portfolio


In [15]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links": row["Links"]},
                       ids = [str(uuid.uuid4())])

In [22]:
links = collection.query(query_texts = job["skills"], n_results =2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [23]:
job = json_res
job['skills']

['Python programming', 'Web scraping']

In [25]:
prompt_email = PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_descrioption}

    ###INSTRUCTION:
    You are Raj, a business development executive at Meta. Meta is a tech giant specializing in social media platforms and technologies. They have expertise in building and managing large-scale social networks, including Facebook, Instagram, Threads, Messenger, and WhatsApp. Their expertise extends to virtual and augmented reality, AI, and data analytics. 

    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Meta's portfolio : {link_list}
    Remember you are Raj, BDE at Meta.
    Do not provide a preamble.
    ###EMAIL(NO PREAMBLE):

    """
    )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description" : str(job), "link_list":links})

"Subject: Expert Python Web Scraping Solutions for Leafpress\n\nDear Hiring Manager at Leafpress,\n\nI came across the job posting for a Software Engineer Intern at Leafpress, and I'm excited to introduce Meta's expertise in building and integrating Python web scrapers. As a business development executive at Meta, I'm confident that our team can assist you in automating utility data collection for large real estate firms.\n\nAt Meta, we have a strong portfolio of Python-based projects, including machine learning and data analytics. Our team has worked on various web scraping projects, and we're well-equipped to handle large-scale data extraction and automation. I'd like to highlight a few relevant examples from our portfolio:\n\n* https://example.com/ml-python-portfolio (Machine Learning with Python)\n* https://example.com/python-portfolio (Python-based Web Development)\n\nOur expertise in Python programming, combined with our experience in web scraping, makes us an ideal fit for your 